# Axisim Punch-Through Shear Test

This example couples two domains via an zero-thickness interface. 

State - 2021-07-19 [RC]:

- Slide34 included with not verified material parameters. The push through test is running for a small step size
- Add the force control boundary conditions in analogy to single facet example
- Change of the parameters to brittle response leads to convergence problems at the transition to inelastic state - check the range which converges.
- Either initial or secant stiffness has been used     

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt

In [16]:
from ibvpy.api import TStepBC , TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

# Material models

In [5]:
from bmcs_matmod.slide.vslide_34 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic

# FE discretization - Displacement-controlled

In [63]:
n_y_e = 10
n_inner_x_e = 5
n_outer_x_e = 5
L_x = 200.0
R_in = 100.0
R_out = 200.0
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=28000, nu=0.3)
m2 = MATS3DElastic(E=28000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_lower.mesh.I[1:-1, -1],
    J=xd_upper.mesh.I[1:-1, 0],
    fets=FETS1D52ULRH()
)

In [64]:
#bond_m = Slide34(E_T=10000, E_N=10000)
material_params = dict(
         E_T=100, gamma_T=200, K_T=0, S_T=0.05, c_T=3, bartau=3, 
         E_N=100, S_N=0.05, c_N = 3, m = 0.15, f_t=5, f_c=80, f_c0 = 40, eta=0.2)
bond_m = Slide34(**material_params)

In [65]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=0.5)
# free_inner_y = BCSlice(slice=xd_inner.mesh[:, 0, :, 0],
#                   var='u', dims=[1], value=0)
bc1 = [axisym, fixed_upper_support, loaded_lower]

In [66]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
    #        'damage': Vis3DStateField(var='omega_a'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.02
s.tstep.fe_domain.serialized_subdomains

In [67]:
#bond_m.interact()

In [68]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 0.02(1), 
	 0.04(1), 
	 0.06(1), 
	 0.08(1), 
	 0.10(1), 
	 0.12(1), 
	 0.14(1), 
	 0.16(1), 
	 0.18(1), 
	 0.20(1), 
	 0.22(1), 
	 0.24(1), 
	 0.26(1), 
	 0.28(1), 
	 0.30(1), 
	 0.32(1), 
	 0.34(1), 
	 0.36(1), 
	 0.38(1), 
	 0.40(1), 
	 0.42(1), 
	 0.44(1), 
	 0.46(1), 
	 0.48(1), 
	 0.50(1), 
	 0.52(1), 
	 0.54(1), 
	 0.56(1), 
	 0.58(1), 
	 0.60(1), 
	 0.62(1), 
	 0.64(1), 
	 0.66(1), 
	 0.68(1), 
	 0.70(1), 
	 0.72(1), 
	 0.74(1), 
	 0.76(1), 
	 0.78(1), 
	 0.80(1), 
	 0.82(1), 
	 0.84(1), 
	 0.86(1), 
	 0.88(1), 
	 0.90(1), 
	 0.92(1), 
	 0.94(1), 
	 0.96(1), 
	 0.98(1), 
	 1.00(1), 



# Plot the punch-through curve

In [69]:
F_to = m.hist.F_t
U_to = m.hist.U_t

In [70]:
F_to.shape

(51, 264)

In [71]:
F_outer_t = np.sum(F_to[:, fixed_upper_support.dofs], axis=-1)
F_inner_t = np.sum(F_to[:, loaded_lower.dofs], axis=-1)
U_inner_top_t = np.average(U_to[:, loaded_lower.dofs], axis=-1)
#U_inner_bot_t = np.average(U_to[:, free_inner_y.dofs], axis=-1)

In [72]:
_, ax = plt.subplots(1,1)
ax.plot(-U_inner_top_t, F_outer_t, label='F_outer(U_top)')
ax.plot(-U_inner_top_t, F_inner_t, label='F_inner(U_top)')
#ax.plot(-U_inner_bot_t, F_inner_t, label='F_inner(U_bot)')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'strain'
    strain_viz = Viz3DTensorField(vis3d=m.hist['strain'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [74]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}

In [75]:
sig_tx = EpsSig_t['sig_pi'].reshape(-1)
tau_tx = EpsSig_t['tau_pi_x'].reshape(-1)
omega_tx = EpsSig_t['omega_T'].reshape(-1)

In [76]:
x_m = xd12.x_Eia[:,:,0].flatten()

In [77]:
_, ax = plt.subplots(1,1)
idx = -1
ax_omega = ax.twinx()
ax.plot(x_m, sig_tx[idx,:], color='blue', label='sig')
ax.plot(x_m, tau_tx[idx,:], color='red', label='tau')
ax_omega.plot(x_m, omega_tx[idx,:], color='red', label='tau')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [78]:
sig_tx

array([ -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.        ,  -0.        ,  -0.        ,  -0.        ,
        -0.99753697,  -0.99718696,  -0.99718696,  -0.99701042,
        -0.99701042,  -0.99693822,  -0.99693822,  -0.99694792,
        -0.99694792,  -0.99703602,  -0.99703602,  -0.99720987,
        -0.99720987,  -0.99749632,  -0.99749632,  -0.99796948,
        -1.99507395,  -1.99437392,  -1.99437392,  -1.99402083,
        -1.99402083,  -1.99387644,  -1.99387644,  -1.99389584,
        -1.99389584,  -1.99407204,  -1.99407204,  -1.99441974,
        -1.99441974,  -1.99499264,  -1.99499264,  -1.99593895,
        -2.99261092,  -2.99156088,  -2.99156088,  -2.99103125,
        -2.99103125,  -2.99081466,  -2.99081466,  -2.99084375,
        -2.99084375,  -2.99110806,  -2.99110806,  -2.99162961,
        -2.99162961,  -2.99248896,  -2.99248896,  -2.99

# FE discretization - Force-controlled

In [12]:
n_y_e = 10
n_inner_x_e = 5
n_outer_x_e = 5
L_x = 200.0
R_in = 100.0
R_out = 200.0
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=28000, nu=0.3)
m2 = MATS3DElastic(E=28000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_lower.mesh.I[1:-1, -1],
    J=xd_upper.mesh.I[1:-1, 0],
    fets=FETS1D52ULRH()
)

In [13]:
#bond_m = Slide34(E_T=10000, E_N=10000)
material_params = dict(
         E_T=100, gamma_T=200, K_T=0, S_T=0.05, c_T=3, bartau=3, 
         E_N=100, S_N=0.05, c_N = 3, m = 0.15, f_t=5, f_c=80, f_c0 = 40, eta=0.2)
bond_m = Slide34(**material_params)

In [14]:
tf_force = TFBilinear()
tf_force.interact()

In [17]:
axisym = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
loaded_lower_slice= BCSlice(slice=xd_lower.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=0.5)
loaded_lower_dofs = loaded_lower_slice.dofs

load = 500
F = load / len(loaded_lower_dofs)

loaded_lower = [BCDof(var='f', dof=dof, value = F, time_function=tf_force  ) 
                     for dof in loaded_lower_dofs]

bc1 = [axisym, fixed_upper_support, loaded_lower]

In [18]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
    #        'damage': Vis3DStateField(var='omega_a'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.02
s.tstep.fe_domain.serialized_subdomains

In [19]:
#bond_m.interact()

In [20]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00

TraitError: Each element of the 'bcond_list' trait of a BCondMngr instance must be an IBCond or None, but a value of [<ibvpy.bcond.bc_dof.BCDof object at 0x000001C9CE6D45E0>, <ibvpy.bcond.bc_dof.BCDof object at 0x000001C9CE6C6D60>, <ibvpy.bcond.bc_dof.BCDof object at 0x000001C9CE6C6BD0>, <ibvpy.bcond.bc_dof.BCDof object at 0x000001C9CE6C6C20>, <ibvpy.bcond.bc_dof.BCDof object at 0x000001C9CEB96E00>, <ibvpy.bcond.bc_dof.BCDof object at 0x000001C9CE6D4590>] <class 'list'> was specified.